In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'
import numpy as np
df = pd.read_csv('../data/SiouxFalls_net.tntp', sep='\t',skiprows=6)
df

In [ ]:
# BPR function
f = np.linspace(0, 20000, 150)
t = 1 * (1 + 0.15*(f/5000))**4
plt.plot(f, t)

In [ ]:
links = pd.read_csv('../data/vl_links.txt', sep='\t',skiprows=0)
links_ab = links[['ANODE', 'BNODE', 'cap_ab', 'LENGTH']]
links_ab.columns = ['init_node', 'term_node', 'capacity', 'free_flow_time']
links_ba = links[['ANODE', 'BNODE', 'cap_ba', 'LENGTH']]
links_ba.columns = ['init_node', 'term_node', 'capacity', 'free_flow_time']
links_code = links_ab.append(links_ba, ignore_index=True)

df_inv = links_code.copy()
df_inv.columns = ['term_node', 'init_node', 'capacity', 'free_flow_time'] # make graph effectively undirected
links_code = links_code.append(df_inv, ignore_index=True)
links_code = links_code[links_code.capacity > 0]

links_code.drop_duplicates(inplace=True)


In [ ]:
px.histogram(links.LENGTH)

In [ ]:
df = pd.read_csv('../data/vl_al.txt', sep='\t',skiprows=0)
df

In [ ]:
px.histogram(links.speed_ab, nbins=20)
#plt.plot(links.cap_ba)

In [ ]:
nodes = pd.read_csv('../data/vl_nodes.txt', sep='\t',skiprows=0)
nodes

In [ ]:
def coords(id):
    n = nodes[nodes.node == id]
    return n.iloc[:,1:].to_numpy()[0]
def dist(id1, id2):
    return ((coords(id1) - coords(id2))**2).sum()**0.5
dist(152522, 50303)
    

In [ ]:
mak = links[(links.STREET=='улица Маковского')] #& (links.lanes_ba > 0)]
anodes, bnodes = mak.ANODE.to_numpy(), mak.BNODE.to_numpy()
acoords = np.array([[coords(id)[0], coords(id)[1]] for id in anodes]).T 
bcoords = np.array([[coords(id)[0], coords(id)[1]] for id in bnodes]).T 

x, y = acoords[0], acoords[1] 
#px.scatter(x=acoords[0], y=acoords[1])

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='markers',
                    name='markers',
                        marker_color='black'))
x, y = bcoords[0], bcoords[1] 
fig.add_trace(go.Scatter(x=x, y=y,
                    mode='markers',
                    name='markers',
                        marker_color='blue'))
fig.add_trace(go.Scatter(x=[coords(86881)[0]], y=[coords(86881)[1]],
                    mode='markers',
                    name='markers',
                        marker_color='magenta'))

In [ ]:
mak[mak.ANODE==86881]

In [ ]:
emp = pd.read_csv('../data/emps.csv', sep='\t')
emp

In [ ]:
emp['post_codes'] = emp.address.apply(lambda x : x.split(',')[0])
emp_pcodes = emp[['post_codes', 'emp']].groupby('post_codes').sum()['emp']
emp_pcodes = pd.to_numeric(emp_pcodes, 'coerce').dropna()
# px.histogram(emp_pcodes, nbins=216)
sns.boxplot(emp_pcodes)

In [ ]:
ans = links.set_index('ANODE').join(nodes.set_index('node'))[['type', 'x', 'y']]
ans['size'] = ((ans['type'] == 'MAJOR') * 5 + (ans['type'] == 'MINOR') * 3 + 1)/5
ans['color'] = -2 #ans['size']
ans.x -= 4 # to distinguish a and b til implement directed edges 
#ans.y -= 1
ans['sym'] = ['triangle-up']*ans.shape[0]


# fig = px.scatter(ans, x='x', y='y', color='color', size='size', text=ans.index)
bns = links.set_index('BNODE').join(nodes.set_index('node'))[['type', 'x', 'y']]
bns['size'] = ((bns['type'] == 'MAJOR') * 5 + (bns['type'] == 'MINOR') * 3 + 1)/5
bns['color'] = -3 #-bns['size']
bns['sym'] = ['triangle-down']*bns.shape[0]
bns.x += 4
#bns.y += 1
# nsize.x = nsize.x / 1e6
# nsize.y = nsize.y / 1e6
ns = ans.append(bns, ignore_index=True)


    
nds = nodes.set_index('node')
x, y, node = [], [], []
xa, xb, ya, yb = [], [], [], []
for i in links_code.index:
    an, bn = links_code.init_node[i], links_code.term_node[i]
    x += [nds.x[an], nds.x[bn], None]
    y += [nds.y[an], nds.y[bn], None]
    xa.append(nds.x[an]); xb.append(nds.x[bn]); ya.append(nds.y[an]); yb.append(nds.y[bn]); 
    node += [an, bn, None]
    
edges = pd.DataFrame({'x':x, 'y':y, 'node':node})
links_code['xa'], links_code['xb'], links_code['ya'], links_code['yb'] = xa, xb, ya, yb


In [ ]:
# try project manually
x0 = np.array([736130.6, 4761298])
y0 = np.array([42.967722, 131.895320])[::-1]
x1 = np.array([763430, 4807718])
y1 = np.array([43.375730, 132.251674])[::-1]
a = (y1 - y0) / (x1 - x0)
# b = y0 - a * x0
# emp['x'] = emp.lat * a[0] - R[0]
# emp['y'] = emp.lon * a[1] - R[1]
R = x0 * a - y0

def to_lon(x):
    return x * a[0] - R[0]

def to_lat(y):
    return y * a[1] - R[1] 
    

In [ ]:
from pyproj import *
inProj = CRS.from_string('EPSG:32652')
outProj = CRS.from_string('epsg:4326')
x1,y1 = -11705274.6374,4826473.6922
proj = Transformer.from_crs(inProj, outProj)
# compare with manual projection
proj.transform(ns.x[0], ns.y[0]), (to_lon(ns.x[0]), to_lat(ns.y[0]))

In [ ]:
del to_lon
del to_lat

In [ ]:
def to_lat_lon(x, y):
    return proj.transform(x, y)
inverse_proj = Transformer.from_crs(outProj, inProj)
def from_lat_lon(lat, lon):
    return inverse_proj.transform(lat, lon)

In [ ]:
edges['lat'], edges['lon'] = to_lat_lon(edges.x, edges.y)
ns['lat'], ns['lon'] = to_lat_lon(ns.x, ns.y) 
nodes['lat'], nodes['lon'] = to_lat_lon(nodes.x, nodes.y)
emp['x'], emp['y'] = from_lat_lon(emp.lat, emp.lon)

In [ ]:
# d_inv = dict(enumerate(emp.district.unique()))
d_inv = {
    0: 'Советский',
    1: 'Первореченский',
    2: 'Ленинский',
    3: np.NaN,
    4: 'Фрунзенский',
    5: 'Первомайский'
}
d = {v: k for k, v in d_inv.items()}
d_inv

In [ ]:
emp['coord'] = emp.lat.apply(str) + " " + emp.lon.apply(str)
emp['dis'] = emp.district.apply(lambda x : d[x])
px.scatter(emp[emp.dis != d[np.NaN]], x='x', y='y', size='emp', color='dis')#, symbol='sym')#, 

In [ ]:
def get_cell(df, lon, lat):
    return df[(df.lon > lon[0]) & (df.lon < lon[1]) & (df.lat > lat[0]) & (df.lat < lat[1])]

emp['coord'] = emp.lat.apply(str) + " " + emp.lon.apply(str)
emp['dis'] = emp.district.apply(lambda x : d[x])
emp_vl = emp[emp.dis != d[np.NaN]]
# emp_f = emp[(emp.lon > 131.5) & (emp.lon < 132.5) & (emp.lat > 42.9) & (emp.lat < 43.5)]
main_cell = [[131.7, 132.4], [42.9, 43.5]]
emp_f = get_cell(emp, *main_cell)

emp_f = emp_f[~((emp_f.lon > 132.2) & (emp_f.lat < 43.2))]
# emp_f = emp_f[emp_f['dis'] == 1]

# px.scatter(emp_vl, x='lon', y='lat', size='emp', color='dis')#, symbol='sym')#, 

In [ ]:
houses = pd.read_csv('../data/housedata.csv')
houses['area'] = pd.to_numeric(houses.AREA, 'coerce')
houses['rooms'] = pd.to_numeric(houses.ROOM_COUNT, 'coerce')
print(houses.area.isna().sum(), houses.rooms.isna().sum())
houses = houses[['LON', 'LAT', 'area']].dropna()
houses.columns = ['lon', 'lat', 'area']
houses = get_cell(houses, *main_cell) 
houses.shape

In [ ]:
borders_lon, borders_lat = [], []
MIN_COUNT = 10
MAX_SUM = 2000
emp_f['value'] = emp_f.emp
def split(df, n, lon, lat):
    global borders_lon, borders_lat
    cells = []
    lons = np.linspace(*lon, n[0]+1)
    lats = np.linspace(*lat, n[1]+1)
    
    for lon in zip(lons[:-1], lons[1:]):
        for lat in zip(lats[:-1], lats[1:]):
            cell_jobs = get_cell(emp_f, lon, lat)
            cell_nodes = get_cell(nodes, lon, lat)
            cell_res = get_cell(houses, lon, lat)
            if cell_jobs.shape[0] > 0:
                
                if cell_jobs.value.sum() > MAX_SUM and cell_jobs.shape[0] > MIN_COUNT:
                    cells += split(cell_jobs, n, lon, lat)
                else:
                    borders_lon += [*lon, *lon[::-1], lon[0], None]
                    borders_lat += [lat[0], lat[0], lat[1], lat[1], lat[0], None]
#                     cell = cell.copy()
#                     cell['color'] = [np.random.random()]*cell.shape[0]
                    cells.append([cell_jobs, cell_nodes, cell_res])
    return cells
        
cells = split(emp_f, (2,2), *main_cell)
# px.line(x=borders_lon, y=borders_lat)

In [ ]:
cells[0]
for cell_job, _, cell_res in cells:
    mass_center = np.array([(cell_job.lat * cell_job.emp).sum(), (cell_job.lon * cell_job.emp).sum()])
    mass_center /= cell_job.emp.sum()
    cell_job.mass_center = mass_center
    
    if cell_res.shape[0] > 0:
        mass_center = np.array([(cell_res.lat * cell_res.area).sum(), (cell_res.lon * cell_res.area).sum()])
        mass_center /= cell_res.area.sum()
        cell_res.mass_center = mass_center
    else:
        cell_res.mass_center = cell_job.mass_center
        
# ns

In [ ]:
%%time
from geopy.distance import geodesic
# TODO use projections

# ndf = ns.drop(['sym', 'type', 'color', 'size'], axis=1).drop_duplicates()
for cell_job, cell_nodes, cell_res in cells:
    for cell in cell_job, cell_res:
        n = cell.mass_center
        dists = cell_nodes.apply(lambda x : geodesic((x.lat, x.lon), (n[0], n[1])).km, axis=1)

        cell.nearest = cell_nodes.iloc[dists.argmin()] if dists.shape[0] > 0 else None
        # Series got by iloc has all fields casted to float for some reason, so to keep int:
        cell.nearest_id = cell_nodes.node.iloc[dists.argmin()] if dists.shape[0] > 0 else None
    
#     print(cell.nearest.shape if not isinstance(cell.nearest, type(None)) else None)
    



In [ ]:
traces = []
# fig = px.scatter(ns, x='lon', y='lat', color='color', size='size', opacity=0.3)#, symbol='sym')#, text=nsize.index)
# traces += [go.Scatter(
#     x=ns.lon,
#     y=ns.lat,
#     mode='markers',
#     marker=dict(size=ns.size, color=ns.color)
#     )]

# traces.append(
#     go.Scatter(
#         x=emp_f.lon,
#         y=emp_f.lat,
#         mode='markers',
#         marker=dict(size=np.log(emp_f.emp.values + 1)*5,
#                 color=emp_f.dis, opacity = 0.7)
#     )
# )
for cell, _, _ in cells:
    traces.append(
        go.Scatter(
            x=cell.lon,
            y=cell.lat,
            mode='markers',
            marker=dict(size=np.log(emp_f.emp.values + 1)*5, # (emp_f.emp.values/emp_f.emp.mean())**0.5*7,
                     opacity = 1)
#             marker=dict(size=np.log(emp_f.emp.values + 1)*5, # (emp_f.emp.values/emp_f.emp.mean())**0.5*7,
        )
    )
for cell, _, _ in cells:
    traces.append(
        go.Scatter(
            x=(cell.mass_center[1],),
            y=(cell.mass_center[0],),
            mode='markers',
            marker=dict(size=5, # (emp_f.emp.values/emp_f.emp.mean())**0.5*7,
                     opacity = 1, color='red')
            
        )
    )
    
# for _, _, cell_res in cells:
#     if not isinstance(cell_res.nearest, type(None)):
#         traces.append(
#             go.Scatter(
#                 x=(cell_res.nearest.lon,),
#                 y=(cell_res.nearest.lat,),
#                 mode='markers',
#                 marker_symbol='square',
#                 marker=dict(size=5, # (emp_f.emp.values/emp_f.emp.mean())**0.5*7,
#                          opacity = 1, color='black')
    
#             )
# )


for cell, _, _ in cells:
    if not isinstance(cell.nearest, type(None)):
        traces.append(
            go.Scatter(
                x=(cell.nearest.lon,),
                y=(cell.nearest.lat,),
                mode='markers',
                marker=dict(size=5, # (emp_f.emp.values/emp_f.emp.mean())**0.5*7,
                         opacity = 1, color='black')
                
            )
        )

# houses

# traces.append(
#     go.Scatter(
#         x=houses.lon,
#         y=houses.lat,
#         mode='markers',
#         marker=dict(size=houses.area/houses.area.mean()*2, opacity = 1, color='red')
#     )
# )

# graph 

traces.append(
    go.Scattergl(
            x=edges.lon,
            y=edges.lat,
            line=dict(color='green'),
# segment colors dont work in plotly   marker=dict(color=np.random.random(len(edges.lon)), cmin=0, cmax=1)
    )
)

# cell borders

traces.append(
    go.Scatter(
            x=borders_lon,
            y=borders_lat,
            line=dict(color='grey')
    )
)

# selected sources/sinks

# traces.append(
#     go.Scatter(
#         x=trips_nodes.lon,
#         y=trips_nodes.lat,
#         mode='markers',
#         marker_symbol='square',
#         marker=dict(size=20,
#                     line=dict(width=5,color='black'),
#                 color=trips_nodes.dis)#, opacity = 0.8)
#     )
# )


# traces.append(go.Scatter(x=(132.038762,),y=(43.354943,), line=dict(color='grey'), mode='markers',
#                 marker_symbol='square',
#                 marker=dict(size=20,color='pink'), opacity=0.5 ))
fig = go.Figure(data=traces)

fig.update_layout(
    autosize=False,
    width=1000,
    height=900,
    paper_bgcolor='white',
    plot_bgcolor='white',
    yaxis_title = 'Широта, градусы',
    xaxis_title = 'Долгота, градусы',
    font = {'size':18} ,
#     showlegend=False,
)

fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = a[0]/a[1],
    range=[43.035, 43.2],
  )
fig.update_xaxes(range=[131.8, 132.1])
fig.update_traces(opacity=0.3, selector=dict(type='scattergl'))
fig.update_traces(text=edges.node, selector=dict(type='scattergl'))
# fig.write_image("network.eps")
# fig.write_image("network.png")
fig.show()


In [ ]:
lon = 131.882, 131.89
lat = 43.117, 43.124
from_lat_lon(lat[0], lon[0])



## Cut testing subnetwork

In [ ]:
xa, xb, ya, yb =[], [], [], []
for i in links.index:
    an, bn = links.ANODE[i], links.BNODE[i]
    xa.append(nds.x[an]); xb.append(nds.x[bn]); ya.append(nds.y[an]); yb.append(nds.y[bn]); 
    node += [an, bn, None]
    
links['xa'], links['xb'], links['ya'], links['yb'] = xa, xb, ya, yb

x0, y0 = from_lat_lon(lat[0], lon[0])
x1, y1 = from_lat_lon(lat[1], lon[1])

links_test = links[links.xa >= x0 ][ links.xb >= x0 ][ links.ya >= y0 ][ links.yb >= y0]
links_test = links_test[links_test.xa <= x1 ][ links_test.xb <= x1 ][ links_test.ya <= y1 ][ links_test.yb <= y1]

emp = emp.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
emp_test = emp[emp.x >= x0 ][ emp.y >= y0][emp.x <= x1 ][ emp.y <= y1 ]
# emp_test.to_csv('../data/emps_test.csv', sep='\t')
# links_test.to_csv('../data/vl_links_test.txt', sep='\t')


# Correspondences by cells

In [ ]:
L, W = {}, {}
res_total = sum(cell_res.area.sum() for _, _, cell_res in cells )
emp_total = sum(cell_emp.emp.sum() for cell_emp, _, _ in cells )
print(emp_total, res_total)
for cell_emp, _, cell_res in cells:
    L[cell_emp.nearest_id] = max(1, cell_emp.emp.sum())
#     W[cell_res.nearest_id] = int(cell_res.area.sum() / res_total * emp_total)
    # use one node for source and sink: division by zero in sinkhorn 
    W[cell_emp.nearest_id] = max(10, int(cell_res.area.sum() / res_total * emp_total))

    
if None in W :
    del W[None]
    del L[None]

for k, v in W.items():
    W[k] -= min(v-10, - sum(L.values()) + sum(W.values()), 100)

print(sum(L.values()), sum(W.values()))

trip_nodes, Ll, Wl = [], [], []
for k in L.keys() | W.keys():
    trip_nodes.append(k)
    Ll.append(L[k] if k in L else 0)
    Wl.append(W[k] if k in W else 0)

    
with open('../data/vl_trips.txt', 'w') as f:
    f.write('trip nodes, L, W\n')
    f.write(' '.join([str(n) for n in trip_nodes]) + '\n')
    f.write(' '.join([str(n) for n in Ll]) + '\n')
    f.write(' '.join([str(n) for n in Wl]) + '\n')
    
    
    

    
    
    
    

In [ ]:
cells[0][0].nearest

In [ ]:
with open('../data/soursinks.csv', 'w') as f:
    f.write('x,y\n')
    for cell_emp, _, cell_res in cells:
        n = cell_emp.nearest
        if not isinstance(n, type(None)):
            f.write(f'{n.x},{n.y}\n')
    

In [ ]:
len(trip_nodes)

In [ ]:
len(L.keys() | W.keys())

# Manual correspondences

In [ ]:
trips_nodes_dict = {
        'Первомайский' : [98866, 7207, 113059, 95578, 40490],
        'Ленинский' : [642953, 33705, 512055, 98893, 500285],
        'Фрунзенский' : [7870, 453482, 4049, 4257, 4306],
        'Первореченский' : [43005, 608219, 567092, 11011, 79983],
        'Советский' : [5949, 12012, 678253, 746461, 768114]
    }

dis = []
x , y = [], []
for key, val in trips_nodes_dict.items():
    dis += [d[key]] * len(val)
    for n in val:
        x.append(nds.x[n])
        y.append(nds.y[n])
trips_nodes = pd.DataFrame({'x':x, 'y':y, 'dis':dis})
trips_nodes['lat'] = to_lat(trips_nodes.y)
trips_nodes['lon'] = to_lon(trips_nodes.x)

In [ ]:
for district in set(emp.district.values):
    print(f'{district}:\t {emp[emp.district.apply(lambda x: x == district)].emp.sum()}') 

In [ ]:
dis_works = {}
for district in set(emp.district.values):
    works = emp[emp.district.apply(lambda x: x == district)].emp.sum()
    # emp.district == None doesnt work    
#     print(f'{district}:\t {works}')
    dis_works[district] = works
dis_works

In [ ]:
dis_residents = {
    "Ленинский": 153_882,
    "Первомайский": 155_072,
    "Первореченский": 145_067,
    "Советский": 92_140,
    "Фрунзенский": 58_740
}

dis_residents_scaled = {}
for key, value in dis_residents.items():
    dis_residents_scaled[key] = int(value / sum(dis_residents.values()) * sum(dis_works.values()))
dis_residents_scaled["Ленинский"] += sum(dis_works.values()) - sum(dis_residents_scaled.values()) 
sum(dis_works.values()), sum(dis_residents_scaled.values())

In [ ]:
trip_nodes, L, W = [], [], []
for district, nodes in trips_nodes_dict.items():
    trip_nodes += nodes
    split = len(nodes)
    
    res = dis_residents_scaled[district]  
    l = [int(res / split)] * split
    l[-1] += res - sum(l)
    L += l
    
    work = dis_works[district]  
    w = [int(work / split)] * split 
    w[-1] += work - sum(w)
    W += w
    
print (trip_nodes, L, W, sep='\n')
# with open('../data/vl_trips.txt', 'w') as f:
#     f.write('trip nodes, L, W\n')
#     f.write(' '.join([str(n) for n in trip_nodes]) + '\n')
#     f.write(' '.join([str(n) for n in L]) + '\n')
#     f.write(' '.join([str(n) for n in W]) + '\n')
    
# for district, target_nodes in trips_nodes_dict.items():
#     target_split = len(target_nodes)
        
    
sum(L), sum(W)
    

In [ ]:
with open('../data/soursinks_man.csv', 'w') as f:
    f.write('x,y\n')
    for n in trip_nodes:
            f.write(f'{nds.x[n]},{nds.y[n]}\n')
    